In [1]:
import pandas as pd

In [15]:
#import data
df=pd.read_csv('sabio_combine_csv.csv',encoding='utf-8')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7808\3222212521.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('sabio_combine_csv.csv',encoding='utf-8')


In [22]:
df.columns

Index(['EntryID', 'Substrate', 'EnzymeType', 'PubMedID', 'Organism',
       'UniprotID', 'ECNumber', 'parameter.type',
       'parameter.associatedSpecies', 'parameter.startValue',
       'parameter.endValue', 'parameter.standardDeviation', 'parameter.unit',
       '<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">', '{'],
      dtype='object')

In [88]:
#del uninterested columns
df_drop=df.drop(columns=['PubMedID','{','<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">','parameter.standardDeviation','parameter.endValue'])
df_drop.columns
print(df_drop.shape)

(314212, 10)


In [104]:
#query interested data
kcat_query=(df_drop['parameter.type']=='kcat')
kcat=df_drop[kcat_query]
print(kcat.shape)
print(kcat['parameter.type'].value_counts())

(27315, 10)
parameter.type
kcat    27315
Name: count, dtype: int64


In [27]:
#observe data distribution of every column
#df['column'].value_counts
#generate the requirement of clean

In [106]:
#clean dataset
clean_unit=(df_drop['parameter.unit']=='s^(-1)')
clean_UniprotID=(df_drop['UniprotID'].notna())&(df['UniprotID'].str.len()==6)
clean_startValue=(df_drop['parameter.startValue'].notna())
kcat_cleaned=kcat[clean_unit&clean_UniprotID&clean_startValue]
print(kcat_cleaned.shape)
#print(kcat_cleaned.info())



C:\Users\Administrator\AppData\Local\Temp\ipykernel_7808\2953537367.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kcat_cleaned=kcat[clean_unit&clean_UniprotID&clean_startValue]


(22238, 10)
(22238, 15)


In [99]:
km_query=(df['parameter.type']=='Km')
km=df_drop[km_query]
print(km.shape)
duplicated_row=km.duplicated(subset=['EntryID',],keep=False)
km=km[~duplicated_row]
print(km.shape)

(54009, 10)
(45933, 10)


In [112]:
#merge
merge_df=pd.merge(kcat_cleaned,km[['EntryID','parameter.associatedSpecies']],on='EntryID',how='inner')
print(merge_df.shape)


(19559, 11)


Index(['EntryID', 'Substrate', 'EnzymeType', 'Organism', 'UniprotID',
       'ECNumber', 'parameter.type', 'parameter.associatedSpecies_x',
       'parameter.startValue', 'parameter.unit',
       'parameter.associatedSpecies_y'],
      dtype='object')

In [133]:
#unqiue check
merge_df.groupby(['EntryID']).size().max()
#merge_df[merge_df.duplicated(subset=['EntryID',],keep=False)]

2

In [136]:
#deduplicate
idx=merge_df.groupby(['EntryID'])[ 'parameter.startValue'].idxmax()
merge_df=merge_df.loc[idx]
merge_df.groupby(['EntryID']).size().max()
print(merge_df.shape)

(19548, 11)


In [137]:
#generate output dataframe
df_output=pd.DataFrame()

In [138]:
df_output['ECNumber']=merge_df['ECNumber']
df_output['Organism']=merge_df['Organism']
df_output['Substrate']=merge_df['parameter.associatedSpecies_y']
df_output['Value']=merge_df['parameter.startValue']
df_output['unit']=merge_df['parameter.unit']
df_output['UniprotID']=merge_df['UniprotID']

In [139]:
df_output

,ECNumber,Organism,Substrate,Value,unit,UniprotID
13768,3.4.21.73,Homo sapiens,Plasminogen,0.405000,s^(-1),P00749
13769,3.4.21.73,Homo sapiens,Plasminogen,0.135000,s^(-1),P00749
2998,1.1.99.31,Pseudomonas putida,(S)-Mandelate,360.000000,s^(-1),P20932
2999,1.1.99.31,Pseudomonas putida,(S)-Mandelate,19.200000,s^(-1),P20932
3000,1.1.99.31,Pseudomonas putida,(S)-Mandelate,2.800000,s^(-1),P20932
...,...,...,...,...,...,...
4511,1.14.14.43,Arabidopsis thaliana,4-Hydroxyphenylacetaldoxime,0.433333,s^(-1),P48421
4512,1.14.14.43,Arabidopsis thaliana,Phenylacetaldoxime,0.416667,s^(-1),P48421
4514,1.14.14.45,Arabidopsis thaliana,3-Indoleacetaldoxime,0.866667,s^(-1),O65782
4515,1.14.14.45,Arabidopsis thaliana,4-Hydroxyphenylacetaldoxime,0.233333,s^(-1),O65782


In [142]:
merge_df.to_csv('sabio_cleaned.csv',encoding='utf-8',index=False)

In [156]:
counts=df_output['Substrate'].value_counts()
for index in counts.index:
    if(len(index))>=6:
        print(index)

4-Nitrophenyl phosphate
Penicillin G
2,2'-Azino-bis(3-ethylbenzthiazoline-6-sulfonic acid)
D-Fructose 1,6-bisphosphate
Acetylthiocholine
L-Tryptophan
Nitrocefin
Syringaldazine
Cephalothin
Dihydrofolate
Ampicillin
D-Glucose
Pyruvate
Cefotaxime
2,6-Dimethoxyphenol
L-Arginine
L-Glutamate
D-Glyceraldehyde 3-phosphate
L-Malate
Cephaloridine
Acetyl-CoA
Imipenem
L-Phenylalanine
Glycerone phosphate
L-Tyrosine
Reduced glutathione
L-Serine
trans,trans-Farnesyl diphosphate
Fumarate
Chorismate
S-Adenosyl-L-methionine
1-Chloro-2,4-dinitrobenzene
L-Glutamine
p-Nitrophenyl-beta-D-glucoside
Acetate
2-Oxoglutarate
L-Aspartate
Isopentenyl diphosphate
Oxaloacetate
D-Xylose
Phosphoenolpyruvate
Isocitrate
Cefuroxime
Angiotensin I
Sulfite
L-Ornithine
D-Fructose 6-phosphate
Glyceraldehyde
5-Bromo-dUMP
o-Nitrophenyl-beta-D-galactoside
p-Nitrophenyl-beta-D-galactoside
2'-(4-Methylumbelliferyl)-alpha-D-N-acetylneuraminic acid
L-Leucine
Prephenate
Ceftazidime
Coenzyme A
L-Lysine
ADPribose
D-Fructose 1-phosphate
